In [ ]:
# import autoreload
# ?autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
from RNN import SimpleRNN
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split

In [ ]:
bid_data = pd.read_csv('dataset/EURCAD_Candlestick_1_M_BID_01.01.2019-07.11.2020.csv')
ask_data = pd.read_csv('dataset/EURCAD_Candlestick_1_M_ASK_01.01.2019-07.11.2020.csv')

bid_data.columns = bid_data.columns + '_bid'
bid_data = bid_data.rename(columns={'Gmt time_bid': 'Gmt time'})

ask_data.columns = ask_data.columns + '_ask'
ask_data = ask_data.rename(columns={'Gmt time_ask': 'Gmt time'})
data = ask_data.join(bid_data.drop('Gmt time', axis=1))
data.head()

In [ ]:
def extract_datetime(dtime):
  """
   receives a timestamp and returns 
   day in the week,
   year, month,  
  """
  d, t = dtime.split(' ')
  ds = np.array(d.split('.'), dtype=int)[::-1]
  ts = np.array(t.split(':'), dtype=float)
  date = datetime(ds[0], ds[1], ds[2])

  return date.weekday(), ds[0], ds[1], ds[2], ts[0], ts[1], ts[2]
  
extract_datetime(bid_data['Gmt time'][970000])

In [ ]:
expanded_datetime_columns = ['weekday', 'year', 'month', 'day', 'hour', 'minute', 'second']
expanded_datetime = np.array([extract_datetime(x) for x in data['Gmt time']], dtype=int)


data = data.drop('Gmt time', axis=1,)
data = pd.DataFrame(expanded_datetime, columns=expanded_datetime_columns).join(data)
data.head()

In [ ]:
data.size

In [ ]:
time_window = 10
label = 'Close_ask'


def create_sequence(data, label, time_window):
    time_window = 10
    label = 'Close_ask'

    size = int(data.shape[0] / time_window) + data.size % time_window

    dataset = np.zeros((size, 10))

    counter = 0
    for i in range(time_window, data.shape[0], time_window):
        dataset[counter] = (data[label].values[i - time_window: i])
        counter += 1
    
    return dataset

dataset = create_sequence(data, label, time_window)
dataset.shape

In [ ]:
X = dataset[:, :time_window-1]
y = dataset[:, 0]

print('X shape: {}, y shape: {}'.format(X.shape, y.shape))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print('train size: {}, test size: {}'.format(X_train.shape, X_test.shape))

In [ ]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_train

In [ ]:
from RNN import SimpleRNN

torch.backends.cudnn.enabled = False
input_size = 1
seq_length = 9
num_layers = 2
hidden_size = 256
learning_rate = 0.001
num_epochs = 10

In [ ]:
model = SimpleRNN(input_size, hidden_size, seq_length, num_layers)
critirion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    for i in range(X_train.shape[0]):
        optimizer.zero_grad()

        scores = model(X_train[i].view(seq_length, 1, -1))
        loss = critirion(scores, y_train[i])

        loss.backward()

        optimizer.step()
        
    
    print(f'epoch: {epoch:3} loss: {loss.item():10.8f}')